In [1]:
# # %load_ext autoreload
# %autoreload 2
import torch
from transformer_vq.nn.attn_vq import VQAttentionQK
from transformer_vq.nn.config_spec import TransformerConfig
from transformer_vq.nn.emb import TransformerEmbedding as Emb
import yaml
from einops import rearrange
with open('conf.yml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
n_vocab = config['n_vocab']
sequence_length = config['sequence_len']
batch_size = config['global_batch_size']
d_model = config['d_model']
config['d_type'] = torch.float32
config['param_dtype'] = torch.float32
config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
data = torch.randint(low=0, high=n_vocab, size=(4, sequence_length))
d_model = config['d_model']
emb = Emb(n_vocab, d_model)
model_config = TransformerConfig(**config)
model = VQAttentionQK(model_config)
data = emb(data)
block_len = config['block_len']
data = rearrange(data, 'b (l s) d -> l b s d', l=block_len)
# data = rearrange(data, 'b s d -> b (s d)')

In [2]:
data.shape

torch.Size([32, 4, 32, 1024])

In [3]:
block_len

32

In [8]:
q, k, v, g = model.compute_k_q_v_g(data)

In [9]:
q.shape

torch.Size([32, 4, 32, 384])

In [11]:
k.shape

torch.Size([32, 4, 3, 32, 128])

In [12]:
v.shape

torch.Size([32, 4, 3, 32, 128])

In [13]:
g.shape

torch.Size([32, 4, 32, 384])

In [5]:
from transformer_vq.nn.norm import LayerNorm

In [6]:
norm = LayerNorm(d_model, 42)

In [7]:
norm

LayerNorm()

In [14]:
import einops; import torch

In [15]:
x = torch.randn(3, 1, 4)
x_squeezed = rearrange(x, 'h 1 w -> h w')

print(x.shape)        # Output: torch.Size([3, 1, 4])
print(x_squeezed.shape)  

torch.Size([3, 1, 4])
torch.Size([3, 4])


In [16]:
import copy
import math

import torch
from torch import nn
import torch.nn.functional as F

# Copy a module N times
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# Implement attention (Scaled Dot Product)
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    attention_result = torch.matmul(p_attn, value)
    return attention_result, p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) 
                             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [19]:
import numpy as np
batch_size = 4
sequence_length = 2048
hidden_size = 128
attention_heads = 1
mha = MultiHeadedAttention(h=attention_heads, d_model=hidden_size)
print("With as many attention queries as there are values:\n")
query = torch.tensor(np.ones([batch_size, 1, hidden_size])).float()
value = torch.tensor(np.ones([batch_size, sequence_length, hidden_size])).float()
result = mha.forward(query, value, value)
print("query:", query.size())
print("value:", value.size())
print("result:", result.size())
print("\n")

With as many attention queries as there are values:

query: torch.Size([4, 1, 128])
value: torch.Size([4, 2048, 128])
result: torch.Size([4, 1, 128])




In [20]:
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    attention_result = torch.matmul(p_attn, value)
    return attention_result, p_attn

In [25]:
queries = torch.randn(4, 2048, 1, 128)
keys = torch.randn(4, 2048, 1, 128)
values = torch.randn(4, 2048, 1, 128)
# shape(batch, sequence_length, num_heads, hidden_size)
attention(queries, keys, values)[0].shape

torch.Size([4, 2048, 1, 128])

In [26]:
pip install flops-profiler

  Obtaining dependency information for flops-profiler from https://files.pythonhosted.org/packages/43/26/5732be586af7ab6cf8a518c91e7a5a44a839aa00d014173eee0398d357c7/flops_profiler-0.1.2-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torchvision.models as models
import torch
from flops_profiler import get_model_profile

with torch.cuda.device(0):
    model = models.alexnet()
    batch_size = 256
    flops, macs, params = get_model_profile(model=model, # model
                                    input_shape=(batch_size, 3, 224, 224), # input shape to the model. If specified, the model takes a tensor with this shape as the only positional argument.
                                    args=None, # list of positional arguments to the model.
                                    kwargs=None, # dictionary of keyword arguments to the model.
                                    print_profile=True, # prints the model graph with the measured profile attached to each module
                                    detailed=True, # print the detailed profile
                                    module_depth=-1, # depth into the nested modules, with -1 being the inner most modules
                                    top_modules=1, # the number of top modules to print aggregated profile
                                    warm_up=10, # the number of warm-ups before measuring the time of each module
                                    as_string=True, # print raw numbers (e.g. 1000) or as human-readable strings (e.g. 1k)
                                    output_file=None, # path to the output file. If None, the profiler prints to stdout.
                                    ignore_modules=None, # the list of modules to ignore in the profiling
                                    func_name='forward') # the function name to profile, "forward" by default, for huggingface generative models, `generate` is used

ImportError: cannot import name 'get_model_profile' from 'flops_profiler' (/Users/jmuneton/miniconda3/envs/torch/lib/python3.9/site-packages/flops_profiler/__init__.py)

In [2]:
import torch
from torchvision.models import resnet18

model = resnet18()
inputs = torch.randn(1, 3, 224, 224)

In [5]:
from torchprofile import profile_macs

macs = profile_macs(model.forward, inputs)

RuntimeError: Cannot insert a Tensor that requires grad as a constant. Consider making it a parameter or input, or detaching the gradient
Tensor:
(1,1,.,.) = 
 0.01 *
  1.1080 -4.0562 -1.3355  1.2117  5.1029  2.2094 -3.2301
  -0.5330 -3.4673  1.4121 -0.6408  1.1193 -0.8178  0.7787
   3.0741 -1.1994 -1.8949  2.7589 -1.7267  1.2214 -2.9912
  -1.7399 -1.8186 -0.3186  2.1653 -1.7571 -0.3243  1.4165
   0.9497 -4.0107 -2.4769  2.7359 -2.2574  0.7575 -1.6913
   1.9700  0.4341  3.1477  0.7139 -2.0988 -1.1568 -4.2788
  -0.8290 -3.1315  1.6158  0.7158 -0.2111 -1.7767 -2.5493

(2,1,.,.) = 
 0.01 *
 -2.7893 -4.6894 -1.8325 -2.6411  3.3986 -3.3846  0.9493
  -1.0141  0.2990  1.1784 -1.6422 -2.2760  1.8405 -1.3625
  -1.7014 -0.9102 -0.4546 -1.2207 -2.3431  0.1905 -1.2274
  -1.9815 -0.7897  3.3881  1.9139  2.2295 -0.3297 -0.3794
  -1.2901  0.5666  3.0023  0.7778  1.6756  1.5022 -2.2762
   0.0174 -2.5352  0.5258 -4.5676 -1.0004  0.4277 -4.4092
  -0.4480 -2.3485  1.3069 -0.8388  0.1634 -1.1821 -0.5386



In [ ]:
import torch
from torchvision.models import resnet50

from thop import profile

model = resnet50()
input = torch.randn(1, 3, 224, 224)
macs, params = profile(model, inputs=(input,))

AttributeError: 'function' object has no attribute 'training'

In [12]:
from torch.profiler import profile, record_function, ProfilerActivity
# Define the method you want to profile
def my_method(input_tensor):
    # Your computation here (example below is just a sample computation)
    output = input_tensor * 2
    output = output @ output.T
    return output

# Create an input tensor
input_tensor = torch.randn(1000, 1000).to('mps')

# Profile the method
with profile(
    activities=[ProfilerActivity.CPU,],
    record_shapes=True,
    profile_memory=True
) as prof:
    with record_function("my_method"):
        my_method(input_tensor)

# Print results
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                    my_method        39.83%     521.000us       100.00%       1.308ms       1.308ms           0 b           0 b             1  
                    aten::mul        49.08%     642.000us        49.46%     647.000us     647.000us           0 b           0 b             1  
                 aten::matmul         0.61%       8.000us         8.56%     112.000us     112.000us           0 b           0 b             1  
                     aten::mm         7.87%     103.000us         7.95%     104.000us     104.000us           0 b           0 b         

STAGE:2024-11-07 20:49:04 79294:3300934 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-11-07 20:49:04 79294:3300934 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-11-07 20:49:04 79294:3300934 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [13]:
from torchvision.models import resnet50
from thop import profile
model = resnet50()
input = torch.randn(1, 3, 224, 224)
macs, params = profile(model, inputs=(input, ))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


In [14]:
macs

4133742592.0

In [19]:
def count_your_model(model, x, y):
    j = model.forward(x)
    return j * y
    # your rule here

input = torch.randn(1, 3, 224, 224)
macs, params = profile(model, inputs=(input, ), 
                        custom_ops={resnet50(): count_your_model})

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


In [38]:
sequence_lengths

[0,
 8192,
 16384,
 24576,
 32768,
 40960,
 49152,
 57344,
 65536,
 73728,
 81920,
 90112,
 98304,
 106496,
 114688,
 122880,
 131072,
 139264,
 147456,
 155648,
 163840,
 172032,
 180224,
 188416,
 196608,
 204800,
 212992,
 221184,
 229376,
 237568,
 245760,
 253952,
 262144,
 270336,
 278528,
 286720,
 294912,
 303104,
 311296,
 319488]

In [ ]:
sequence_lengths = [1024 * 2**i for i in range(0,35)]

In [49]:
sequence_lengths

[1024,
 9216,
 17408,
 25600,
 33792,
 41984,
 50176,
 58368,
 66560,
 74752,
 82944,
 91136,
 99328,
 107520,
 115712,
 123904,
 132096,
 140288,
 148480,
 156672,
 164864,
 173056,
 181248,
 189440,
 197632,
 205824,
 214016,
 222208,
 230400,
 238592,
 246784,
 254976,
 263168,
 271360,
 279552]

In [ ]:
from torchvision.models import resnet50
from thop import profile
model = resnet50()
input = torch.randn(1, 3, 224, 224)
macs, params = profile(model, inputs=(input, ))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
